# Assignment problem

- Two sets, where the goal is to match objects in sets and optimize some decision
- Constraints ensure that each object is matched only once
- This is a special case of a transportation problem where all supply and demand are 1
- Since it is a transportation problem, it will have an integer solution

## Problem description

*This problem is from the MSBA Optimization course.*

There are 4 pieces of land and 6 bidders. Each bidder wants at most one piece of land. Which bids should be accepted to maximize revenue?

## Mathematical formulation

- Zones $i$
- Buyers $j$
- b_{i,j}: Bid for zone i by buyer j

**Variables**

- $x_{i,j}$: 1 if bid for zone $i$ is accepted from buyer $j$, 0 otherwise

**Objective**

Maximize the accepted bid amount.

$\sum\nolimits_{i} \sum\nolimits_{j} b_{i,j} \cdot x_{i,j}$

**Constraints**

Every zone must be sold only once (1). Each bidder can win at most one zone (2).

1. $\sum\nolimits_{j} x_{i,j} = 1 \, \forall i$
2. $\sum\nolimits_{i} x_{i,j} \leq 1 \, \forall j$

## PuLP model

In [1]:
# Assignment Problem: data
Zones = ['Zone 1', 'Zone 2', 'Zone 3', 'Zone 4']
Bidders = ['A', 'B', 'C', 'D', 'E', 'F']

Bids = {
    ('Zone 1', 'A') : 17.7,	('Zone 1', 'B') : 17.8,	('Zone 1', 'C') : 18.1,	('Zone 1', 'D') : 17.9,	('Zone 1', 'E') : 17.6,	('Zone 1', 'F') : 17.4,
    ('Zone 2', 'A') : 16.3,	('Zone 2', 'B') : 17.2,	('Zone 2', 'C') : 17.4,	('Zone 2', 'D') : 16.8,	('Zone 2', 'E') : 17.3,	('Zone 2', 'F') : 16.4,
    ('Zone 3', 'A') : 17.8,	('Zone 3', 'B') : 17.6,	('Zone 3', 'C') : 18.5,	('Zone 3', 'D') : 16.9,	('Zone 3', 'E') : 18.3,	('Zone 3', 'F') : 18,
    ('Zone 4', 'A') : 17.1,	('Zone 4', 'B') : 18,	('Zone 4', 'C') : 18.3,	('Zone 4', 'D') : 18,	('Zone 4', 'E') : 18.1,	('Zone 4', 'F') : 17.6
}

In [51]:
import pulp

In [52]:
## Create model as maximization problem
mdl = pulp.LpProblem("BidAssignment", pulp.LpMaximize)

In [53]:
## Create the matrix assignment variable
assign = pulp.LpVariable.dicts("Bids", ((i, j) for i in Zones for j in Bidders),
                              lowBound = 0,
                              upBound = 1)

In [54]:
## Maximize the accepted bid amount
mdl += pulp.lpSum([assign[i,j] * Bids[i,j] for i, j in assign])

In [55]:
## Constraint 1: Each zone sold once
for i in Zones:
    cname = "Zone sold once "+i
    mdl += (
        pulp.lpSum([assign[i,j] for j in Bidders]) == 1,
        cname,
    )

In [56]:
## Constraint 2: Each bidder only sold at most one zone
for j in Bidders:
    cname = "Bidder wins at most once "+j
    mdl += (
        pulp.lpSum([assign[i,j] for i in Zones]) <= 1,
        cname,
    )

In [58]:
mdl.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/smontgom/opt/anaconda3/envs/opt/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/w1/99fv6vmj5h5dv7xr2y9jg_3c0000gn/T/6a52a918ccdc445283dc1ef58d8fdc44-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/w1/99fv6vmj5h5dv7xr2y9jg_3c0000gn/T/6a52a918ccdc445283dc1ef58d8fdc44-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 88 RHS
At line 99 BOUNDS
At line 124 ENDATA
Problem MODEL has 10 rows, 24 columns and 48 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 10 (0) rows, 24 (0) columns and 48 (0) elements
0  Obj -0 Primal inf 3.9999996 (4) Dual inf 422.1 (24)
8  Obj 71.7
Optimal - objective value 71.7
Optimal objective 71.7 - 8 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.

1

## Solution report

In [63]:
max_revenue = pulp.value(mdl.objective)
print("Max revenue: {:.2f}".format(max_revenue))

Max revenue: 71.70


In [66]:
for i in Zones:
     for j in Bidders:
            if (pulp.value(assign[i,j]) == 1):
                print("{:s} bid accepted: {:s}".format(i,j))

Zone 1 bid accepted: D
Zone 2 bid accepted: B
Zone 3 bid accepted: E
Zone 4 bid accepted: C
